# Infer Dark Ion Channel Function from Clustering
This notebook will infer 'dark' Ion Channel (IC) function based on hierarchical clustering with kinases that have known functions. We will cluster ICs based on their expression in the CCLE. We will begin by importing required libraries and defining a few functions for later use. 

In [1]:
import pandas as pd
from clustergrammer_widget import *
from scipy.spatial.distance import pdist, squareform
net = Network()

In [2]:
gene_info = net.load_json_to_dict('../grant_pois/gene_info_with_dark.json')
net.load_file('../hzome_data/my_CCLE_exp.txt')
ccle = net.export_df()

In [3]:
def add_cats(gene_class, genes):
    ''' Add categories to list of genes '''    
    gene_title = class_titles[gene_class]
    genes_cat = []
    for inst_gene in genes:
        inst_tuple = ()
        inst_name = gene_title + ': ' + inst_gene
        if inst_gene in gene_info[gene_class]['dark']:
          inst_cat = 'Dark Gene: True'
        else:
          inst_cat = 'Dark Gene: False'
        inst_tuple = (inst_name, inst_cat)
        genes_cat.append( inst_tuple )
    return genes_cat

def filter_genes(ccle, gene_class):
    ''' Filter DataFrame to selected gene class '''
    ccle = ccle.transpose()
    all_genes = ccle.columns.tolist()
    all_gene_class = gene_info[gene_class]['all']
    found_genes = sorted(list(set(all_genes).intersection(all_gene_class)))
    ccle_filt = ccle[found_genes]
    ccle_filt = ccle_filt.transpose()
    return ccle_filt

class_titles = {}
class_titles['KIN'] = 'Kinases'
class_titles['IC'] = 'Ion Channels'
class_titles['GPCR'] = 'GPCRs'

### Filter for ICs
We will first filter the CCLE gene expression data for ICs. Then we will calculate a IC-IC similarity matrix based on CCLE expression and visualizze it using Clustergrammer. 

In [4]:
ccle_ic = filter_genes(ccle, 'IC')
net.load_df(ccle_ic)

### Calculate Similarity Matrix

In [6]:
net.normalize(axis='row', norm_type='zscore', keep_orig=False)
ccle_ic = net.export_df()

In [7]:
inst_dm = pdist(ccle_ic, metric='cosine')
inst_dm = squareform(inst_dm)
inst_dm = 1 - inst_dm
sim_cutoff = 0.15
inst_dm[ abs(inst_dm) < sim_cutoff] = 0

### Visualize Similarity Matrix

In [9]:
genes = ccle_ic.index.tolist()
genes_cat = add_cats('IC', genes)
df_dm = pd.DataFrame(data=inst_dm, columns=genes_cat, index=genes_cat)
net.load_df(df_dm)
net.make_clust(views[])
clustergrammer_widget(network=net.widget())

We see about six well-defined clusters of IC with similar expresssion across the CCLE. I have included Enrichr links for each of the six clusters. Enrichment for terms from the KEGG 2016 and Wikipathways 2016 libraries appeared to show distinct functions for some of these clusters. For instance, Cluster-1 showed enrichment for Cardiac and muscle contraction pathways (see link 1) and Cluster-3 shoed enrichment for addiction related pathways (e.g Nicotine addiction, Serotonergic/Dopaminergic synapse function, and Hypothetical Network for Drug addiction). 

## Enrichr Links for Clusters identified from IC-CCLE Clustering
* 1 https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=1botj
* 2 https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=1botl
* 3 https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=1botm
* 4 https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=1bq99
* 5 https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=1botn
* 6 https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=1boto




We are going to focus on the cluster in the middle of the heatmap (with 49 genes). Below we have extracted the names of these ICs from this cluster (using the interactive dendrogram):

In [11]:
clust_names = ["ANO5", "BEST3", "CACNA1A", "CACNA1B", "CACNA1D", "CACNA1E", "CACNA2D1", "CACNA2D2", "CACNB2", "CACNB3", "CHRNA1", "CHRNA3", "CLCN5", "CNGA3", "GABRA5", "GABRB2", "GABRB3", "GABRG2", "GRIA1", "GRIA2", "GRIA4", "GRIK3", "GRIN3A", "HCN1", "HCN3", "HTR3A", "KCNA1", "KCNB1", "KCNB2", "KCNC1", "KCND2", "KCNG3", "KCNH2", "KCNH5", "KCNH6", "KCNH7", "KCNH8", "KCNIP4", "KCNJ11", "KCNJ3", "KCNJ6", "KCNK10", "KCNK3", "KCNT2", "SCN1A", "SCN2A", "SCN3A", "SCN3B", "SCN8A"]
clust_names_cat = add_cats('IC', clust_names)

df_dm = df_dm[clust_names_cat]
df_dm = df_dm.transpose()
df_dm = df_dm[clust_names_cat]

net.load_df(df_dm)
net.make_clust(views=[])
clustergrammer_widget(network=net.widget())